In [1]:
# import the necessary packages
from sklearn.metrics import classification_report
from keras.datasets import cifar10
import autokeras as ak
import os
import datetime
import torch
from autokeras.utils import pickle_from_file
from keras.models import load_model
from thop import profile    #pip install --upgrade git+https://github.com/Lyken17/pytorch-OpCounter.git
from keras.models import model_from_json

Using TensorFlow backend.


In [2]:
print(ak.constant.Constant.BACKEND)

torch


In [3]:
def runMain():
    # load the training and testing data, then scale it into the
    # range [0, 1]
    print(datetime.datetime.now())
    print("[INFO] loading CIFAR-10 data...")
    ((trainX, trainY), (testX, testY)) = cifar10.load_data()
    trainX = trainX.astype("float") / 255.0
    testX = testX.astype("float") / 255.0

    # initialize the label names for the CIFAR-10 dataset
    labelNames = ["airplane", "automobile", "bird", "cat", "deer",
        "dog", "frog", "horse", "ship", "truck"]

    # initialize the output directory
    OUTPUT_PATH = "output"

    seconds= 12*60*60 #12 hour
    print(datetime.datetime.now())
    print("[INFO] training model for {} seconds max...".format(seconds))
    model = ak.ImageClassifier(path=os.path.join(os.getcwd(),"automodels"),verbose=True, 
                               augment=True) #, searcher_args={'trainer_args':{'max_iter_num':10}})
    model.fit(trainX, trainY, time_limit=seconds)
    print(datetime.datetime.now())
    #saveModel(model)
    # print("[INFO] final_fit")
    # model.final_fit(trainX, trainY, testX, testY, retrain=True)

    # evaluate the Auto-Keras model
    # score = model.evaluate(testX, testY)
    # predictions = model.predict(testX)
    # report = classification_report(testY, predictions,target_names=labelNames)

# write the report to disk
# p = os.path.join(OUTPUT_PATH, "{}.txt".format(seconds))
# f = open(p, "w")
# f.write(report)
# f.write("\nscore: {}".format(score))
# f.close()

In [4]:
def get_flops(model):
    return profile(model, input_size=(1, 3, 32,32))

In [5]:
def getModule():
    path = os.path.join('automodels', 'module')
    cnn_module = pickle_from_file(path)
    return cnn_module

In [6]:
def saveModel(model):
    # serialize model to JSON
    model_json = model.to_json()
    with open("model.json", "w") as json_file:
        json_file.write(model_json)
    # serialize weights to HDF5
    model.save_weights("model.h5")
    print("Saved model to disk")

In [7]:
def getModelData(module):
    with open("results.txt", "a") as resfile:
        resfile.write("model Id,total params,file size,flops,params from profile,metric,graph size\n")
    for item in module.searcher.history:  
        model_id = item['model_id']
        graph = module.searcher.load_model_by_id(model_id)
        model = graph.produce_model()
        print(model)
        flops, params = get_flops(model)
        file_path=os.path.join('automodels', "{}.pt".format(model_id))
        torch.save(model.state_dict(), file_path)
        pytorch_total_params = sum(p.numel() for p in model.parameters())
        file_size = os.path.getsize(file_path)
        print("\nmodel Id {}:\n   total params = {}\n   file size = {} kB\n   flops: {}\n   params from profile: {}".format(model_id,pytorch_total_params, file_size/1000, flops, params))
        with open("results.txt", "a") as resfile:
            resfile.write("{},{},{},{},{},{},{}\n".format(model_id,pytorch_total_params, file_size/1000, flops, params,item['metric_value'],graph.size()))

In [ ]:
if __name__ == "__main__":
    runMain()

2019-07-18 19:00:26.109902
[INFO] loading CIFAR-10 data...
2019-07-18 19:00:27.993851
[INFO] training model for 43200 seconds max...
Saving Directory: /home/Tal/AutoDLProject/AutoDLProject/automodels
Preprocessing the images.
Preprocessing finished.

Initializing search.
Initialization finished.


+----------------------------------------------+
|               Training model 0               |
+----------------------------------------------+
                                                                                                    
No loss decrease after 5 epochs.


Saving model.
+--------------------------------------------------------------------------+
|        Model ID        |          Loss          |      Metric Value      |
+--------------------------------------------------------------------------+
|           0            |   4.467973697185516    |         0.6088         |
+--------------------------------------------------------------------------+


+----------------



+----------------------------------------------+
|              Training model 12               |
+----------------------------------------------+
                                                                                                    
No loss decrease after 5 epochs.


Saving model.
+--------------------------------------------------------------------------+
|        Model ID        |          Loss          |      Metric Value      |
+--------------------------------------------------------------------------+
|           12           |   2.9871159553527833   |         0.7308         |
+--------------------------------------------------------------------------+


+----------------------------------------------+
|              Training model 13               |
+----------------------------------------------+
                                                                                                    
No loss decrease after 5 epochs.


Saving model.
+----------------



+----------------------------------------------+
|              Training model 24               |
+----------------------------------------------+
                                                                                                    
No loss decrease after 5 epochs.


Saving model.
+--------------------------------------------------------------------------+
|        Model ID        |          Loss          |      Metric Value      |
+--------------------------------------------------------------------------+
|           24           |   2.6118838667869566   |         0.7684         |
+--------------------------------------------------------------------------+


+----------------------------------------------+
|              Training model 25               |
+----------------------------------------------+
                                                                                                    
No loss decrease after 5 epochs.


Saving model.
+----------------



+----------------------------------------------+
|              Training model 36               |
+----------------------------------------------+
Epoch-11, Current Metric - 0.758:   8%|█▊                      | 30/387 [00:06<01:00,  5.91 batch/s]

In [ ]:
getModelData(getModule())